In [3]:
import pandas as pd

DATA_PATH = "data/data.csv"

df = pd.read_csv(DATA_PATH)
df.head()


,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country,price
0,2014-05-02 00:00:00,3.0,1.50,1340,7912,1.5,0,0,3.0,1340,0,1955.0,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA,313000.0
1,2014-05-02 00:00:00,5.0,2.50,3650,9050,2.0,0,4,5.0,3370,280,1921.0,0,709 W Blaine St,Seattle,WA 98119,USA,2384000.0
2,2014-05-02 00:00:00,3.0,2.00,1930,11947,1.0,0,0,4.0,1930,0,1966.0,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA,342000.0
3,2014-05-02 00:00:00,3.0,2.25,2000,8030,1.0,0,0,4.0,1000,1000,1963.0,0,857 170th Pl NE,Bellevue,WA 98008,USA,420000.0
4,2014-05-02 00:00:00,4.0,2.50,1940,10500,1.0,0,0,4.0,1140,800,1976.0,1992,9105 170th Ave NE,Redmond,WA 98052,USA,550000.0


In [21]:
# Выводим структурные сведенья о DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           4600 non-null   object 
 1   bedrooms       4600 non-null   float64
 2   bathrooms      4600 non-null   float64
 3   sqft_living    4600 non-null   int64  
 4   sqft_lot       4600 non-null   int64  
 5   floors         4600 non-null   float64
 6   waterfront     4600 non-null   int64  
 7   view           4600 non-null   int64  
 8   condition      4520 non-null   float64
 9   sqft_above     4600 non-null   int64  
 10  sqft_basement  4600 non-null   int64  
 11  yr_built       4390 non-null   float64
 12  yr_renovated   4600 non-null   int64  
 13  street         4600 non-null   object 
 14  city           4589 non-null   object 
 15  statezip       4600 non-null   object 
 16  country        4600 non-null   object 
 17  price          4600 non-null   float64
dtypes: float

Структура следующая:
date – дата продажи
bedrooms – количество спален
bathrooms – количество ванных комнат
sqft_living – жилая площадь
sqft_lot – общая площадь участка
floors – количество этажей
waterfront – наличие выхода к воде
view – оценка вида
condition – оценка состояния дома
sqft_above – площадь над уровнем земли
sqft_basement – площадь под уровнем земли
yr_built – год постройки дома
yr_renovated – год последнего ремонта
street – название улицы
city – город
statezip – комбинация штата и ZIP-кода
country – страна

In [28]:
# Выводит статистику по столбцам
df.describe(include='all')


,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country,price
count,4600,4600.000000,4600.000000,4600.000000,4.600000e+03,4600.000000,4600.000000,4600.000000,4520.000000,4600.000000,4600.000000,4390.000000,4600.000000,4600,4589,4600,4600,4.600000e+03
unique,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4525,44,77,1,NaN
top,2014-06-23 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2520 Mulberry Walk NE,Seattle,WA 98103,USA,NaN
freq,142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1571,148,4600,NaN
mean,NaN,3.400870,2.160815,2139.346957,1.485252e+04,1.512065,0.007174,0.240652,3.450664,1827.265435,312.081522,1970.828929,808.608261,NaN,NaN,NaN,NaN,5.519630e+05
std,NaN,0.908848,0.783781,963.206916,3.588444e+04,0.538288,0.084404,0.778405,0.678601,862.168977,464.137228,29.736162,979.414536,NaN,NaN,NaN,NaN,5.638347e+05
min,NaN,0.000000,0.000000,370.000000,6.380000e+02,1.000000,0.000000,0.000000,1.000000,370.000000,0.000000,1900.000000,0.000000,NaN,NaN,NaN,NaN,0.000000e+00
25%,NaN,3.000000,1.750000,1460.000000,5.000750e+03,1.000000,0.000000,0.000000,3.000000,1190.000000,0.000000,1951.000000,0.000000,NaN,NaN,NaN,NaN,3.228750e+05
50%,NaN,3.000000,2.250000,1980.000000,7.683000e+03,1.500000,0.000000,0.000000,3.000000,1590.000000,0.000000,1976.000000,0.000000,NaN,NaN,NaN,NaN,4.609435e+05
75%,NaN,4.000000,2.500000,2620.000000,1.100125e+04,2.000000,0.000000,0.000000,4.000000,2300.000000,610.000000,1997.000000,1999.000000,NaN,NaN,NaN,NaN,6.549625e+05


Числовые признаки:
count - количество не пустых значений,
mean - среднее арифметическое,
std - стандартное отклонение,
min, 25%, 50%, 75%, max - значения.

Категориальные/строковые признаки:
unique - количество уникальных элементов,
top - самое частое значение,
freq - частота самого частого значения.

Заметки по данным:
date – строковый признак, требует преобразования в datetime (преобразуем)
bedrooms – числовой признак, распределён нормально (оставляем как есть)
bathrooms – числовой признак, распределён нормально (оставляем как есть)
sqft_living – числовой признак, скошено вправо (применяем log)
sqft_lot – числовой признак, сильно скошено вправо (применяем log)
floors – дискретный числовой признак, нормальная форма (оставляем как есть)
waterfront – бинарный признак 0/1 (оставляем как есть)
view – дискретный рейтинговый признак 0–4 (оставляем как есть)
condition – дискретный рейтинговый признак 1–5 (оставляем как есть)
sqft_above – числовой признак, скошено вправо (применяем log)
sqft_basement – числовой признак, скошено вправо (применяем log)
yr_built – числовой признак, монотонный диапазон (преобразуем в возраст)
yr_renovated – числовой признак, почти всегда 0 (преобразуем в бинарный ремонт)
street – категориальный признак высокой кардинальности, значения почти уникальны (не используем)
city – категориальный признак высокой кардинальности (не используем)
statezip – категориальный признак высокой кардинальности (не используем)
country – константа, одно значение (не используем)

In [29]:
# Показывает количество пропусков в каждом столбце
df.isna().sum()

date               0
bedrooms           0
bathrooms          0
sqft_living        0
sqft_lot           0
floors             0
waterfront         0
view               0
condition         80
sqft_above         0
sqft_basement      0
yr_built         210
yr_renovated       0
street             0
city              11
statezip           0
country            0
price              0
dtype: int64